In [1]:
!pip install git+git://github.com/geophysics-ubonn/reda

  Cloning git://github.com/geophysics-ubonn/reda to /tmp/pip-q7z8_d4o-build


In [2]:
import reda.utils.mpl
plt, mpl = reda.utils.mpl.setup()

import numpy as np
np.random.seed(2017)

from reda import ERT
obj = ERT()
import glob
for nr, filename in enumerate(sorted(
        glob.glob('data/pygimli_*.ohm'))):
    obj.import_bert(filename, timestep=nr)

Summary:
                A           B           M           N      K       rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  741.000000   
mean    13.333333   14.333333   27.666667   28.666667   -1.0    1.265686   
std      9.073772    9.073772    9.073772    9.073772    0.0    3.797820   
min      1.000000    2.000000    3.000000    4.000000   -1.0    0.000766   
25%      6.000000    7.000000   21.000000   22.000000   -1.0    0.007051   
50%     12.000000   13.000000   29.000000   30.000000   -1.0    0.036854   
75%     20.000000   21.000000   35.000000   36.000000   -1.0    0.275161   
max     38.000000   39.000000   40.000000   41.000000   -1.0   17.875655   

       valid           R  timestep  
count  741.0  741.000000     741.0  
mean     1.0   -1.265686       0.0  
std      0.0    3.797820       0.0  
min      1.0  -17.875655       0.0  
25%      1.0   -0.275161       0.0  
50%      1.0   -0.036854       0.0  
75%      1.0   -0.007051       0.0  
max      1

                A           B           M           N      K       rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  741.000000   
mean    13.333333   14.333333   27.666667   28.666667   -1.0    1.232090   
std      9.073772    9.073772    9.073772    9.073772    0.0    3.820308   
min      1.000000    2.000000    3.000000    4.000000   -1.0    0.000003   
25%      6.000000    7.000000   21.000000   22.000000   -1.0    0.000254   
50%     12.000000   13.000000   29.000000   30.000000   -1.0    0.013319   
75%     20.000000   21.000000   35.000000   36.000000   -1.0    0.228072   
max     38.000000   39.000000   40.000000   41.000000   -1.0   18.042386   

       valid           R  timestep  
count  741.0  741.000000     741.0  
mean     1.0   -1.232090      12.0  
std      0.0    3.820308       0.0  
min      1.0  -18.042386      12.0  
25%      1.0   -0.228072      12.0  
50%      1.0   -0.013319      12.0  
75%      1.0   -0.000254      12.0  
max      1.0   -0.0

max      1.0   -0.000003      19.0  
merging with existing data
Summary:
                A           B           M           N      K       rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  741.000000   
mean    13.333333   14.333333   27.666667   28.666667   -1.0    1.202877   
std      9.073772    9.073772    9.073772    9.073772    0.0    3.830614   
min      1.000000    2.000000    3.000000    4.000000   -1.0    0.000003   
25%      6.000000    7.000000   21.000000   22.000000   -1.0    0.000042   
50%     12.000000   13.000000   29.000000   30.000000   -1.0    0.001086   
75%     20.000000   21.000000   35.000000   36.000000   -1.0    0.152524   
max     38.000000   39.000000   40.000000   41.000000   -1.0   18.103418   

       valid           R  timestep  
count  741.0  741.000000     741.0  
mean     1.0   -1.202877      20.0  
std      0.0    3.830614       0.0  
min      1.0  -18.103418      20.0  
25%      1.0   -0.152524      20.0  
50%      1.0   -0.00

merging with existing data
Summary:
                A           B           M           N      K         rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  7.410000e+02   
mean    13.333333   14.333333   27.666667   28.666667   -1.0  1.171571e+00   
std      9.073772    9.073772    9.073772    9.073772    0.0  3.831912e+00   
min      1.000000    2.000000    3.000000    4.000000   -1.0  3.612317e-07   
25%      6.000000    7.000000   21.000000   22.000000   -1.0  3.190903e-06   
50%     12.000000   13.000000   29.000000   30.000000   -1.0  2.316959e-04   
75%     20.000000   21.000000   35.000000   36.000000   -1.0  8.555264e-02   
max     38.000000   39.000000   40.000000   41.000000   -1.0  1.797993e+01   

       valid             R  timestep  
count  741.0  7.410000e+02     741.0  
mean     1.0 -1.171571e+00      33.0  
std      0.0  3.831912e+00       0.0  
min      1.0 -1.797993e+01      33.0  
25%      1.0 -8.555264e-02      33.0  
50%      1.0 -2.316959e-04 

                A           B           M           N      K         rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  7.410000e+02   
mean    13.333333   14.333333   27.666667   28.666667   -1.0  1.150840e+00   
std      9.073772    9.073772    9.073772    9.073772    0.0  3.810371e+00   
min      1.000000    2.000000    3.000000    4.000000   -1.0  2.296891e-07   
25%      6.000000    7.000000   21.000000   22.000000   -1.0  1.534639e-06   
50%     12.000000   13.000000   29.000000   30.000000   -1.0  1.263760e-04   
75%     20.000000   21.000000   35.000000   36.000000   -1.0  6.226777e-02   
max     38.000000   39.000000   40.000000   41.000000   -1.0  1.797993e+01   

       valid             R  timestep  
count  741.0  7.410000e+02     741.0  
mean     1.0 -1.150840e+00      46.0  
std      0.0  3.810371e+00       0.0  
min      1.0 -1.797993e+01      46.0  
25%      1.0 -6.226777e-02      46.0  
50%      1.0 -1.263760e-04      46.0  
75%      1.0 -1.534639e-

Summary:
                A           B           M           N      K         rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  7.410000e+02   
mean    13.333333   14.333333   27.666667   28.666667   -1.0  1.146691e+00   
std      9.073772    9.073772    9.073772    9.073772    0.0  3.801749e+00   
min      1.000000    2.000000    3.000000    4.000000   -1.0  1.717197e-07   
25%      6.000000    7.000000   21.000000   22.000000   -1.0  1.892424e-06   
50%     12.000000   13.000000   29.000000   30.000000   -1.0  1.265724e-04   
75%     20.000000   21.000000   35.000000   36.000000   -1.0  7.647600e-02   
max     38.000000   39.000000   40.000000   41.000000   -1.0  1.796941e+01   

       valid             R  timestep  
count  741.0  7.410000e+02     741.0  
mean     1.0 -1.146691e+00      59.0  
std      0.0  3.801749e+00       0.0  
min      1.0 -1.796941e+01      59.0  
25%      1.0 -7.647600e-02      59.0  
50%      1.0 -1.265724e-04      59.0  
75%      1.0 -1

Summary:
                A           B           M           N      K         rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  7.410000e+02   
mean    13.333333   14.333333   27.666667   28.666667   -1.0  1.161646e+00   
std      9.073772    9.073772    9.073772    9.073772    0.0  3.791170e+00   
min      1.000000    2.000000    3.000000    4.000000   -1.0  3.587424e-07   
25%      6.000000    7.000000   21.000000   22.000000   -1.0  3.562633e-06   
50%     12.000000   13.000000   29.000000   30.000000   -1.0  2.869739e-04   
75%     20.000000   21.000000   35.000000   36.000000   -1.0  1.044468e-01   
max     38.000000   39.000000   40.000000   41.000000   -1.0  1.796406e+01   

       valid             R  timestep  
count  741.0  7.410000e+02     741.0  
mean     1.0 -1.161646e+00      73.0  
std      0.0  3.791170e+00       0.0  
min      1.0 -1.796406e+01      73.0  
25%      1.0 -1.044468e-01      73.0  
50%      1.0 -2.869739e-04      73.0  
75%      1.0 -3

merging with existing data
Summary:
                A           B           M           N      K       rho_a  \
count  741.000000  741.000000  741.000000  741.000000  741.0  741.000000   
mean    13.333333   14.333333   27.666667   28.666667   -1.0    1.180185   
std      9.073772    9.073772    9.073772    9.073772    0.0    3.786346   
min      1.000000    2.000000    3.000000    4.000000   -1.0    0.000001   
25%      6.000000    7.000000   21.000000   22.000000   -1.0    0.000009   
50%     12.000000   13.000000   29.000000   30.000000   -1.0    0.001438   
75%     20.000000   21.000000   35.000000   36.000000   -1.0    0.155060   
max     38.000000   39.000000   40.000000   41.000000   -1.0   17.887017   

       valid           R  timestep  
count  741.0  741.000000     741.0  
mean     1.0   -1.180185      88.0  
std      0.0    3.786346       0.0  
min      1.0  -17.887017      88.0  
25%      1.0   -0.155060      88.0  
50%      1.0   -0.001438      88.0  
75%      1.0   -0.00